## AD ML binary classification

"Supervised Machine Learning (ML) algorithms were used to identify the discriminative power of the top ten differentially expressed genes in distinguishing between the two categories. Support vector machines using both linear and radial basis function kernels were selected for the binary classification in addition to random forest and quadratic Bayes algorithms [36]. The data was divided into a training set and a test set. The training set consisted of approximately 70% of the samples (114 samples) and the remaining 30% were used for testing to test and approximate how the classifiers generalize to unknown data. There was no overlap in the subjects and samples between the training and testing data to avoid any correlation between samples in both sets. The genes were used as features for each sample with all different combinations of N genes (2 ≤ N ≤ 10) out of 10 genes. For every combination of N genes, the combination with the maximum accuracy on the training data was chosen and the corresponding accuracy on the test data was reported on those N genes to avoid data snooping. Precision and recall scores in addition to the F1 score were reported for the testing accuracy that corresponded to the maximum training accuracy over all combinations."

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7717689/



This version keeps **ALL** patients and samples and does **NOT** exclude patients with TBI

In [1]:
import os
import pickle as pkl
import pandas as pd
import numpy as np
import random
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE


## Data Prep

In [6]:
samples = pd.read_csv('../data/raw/gene_expression_matrix_2016-03-03/columns-samples.csv')
donor_info = pd.read_csv('../data/raw/DonorInformation.csv')


In [7]:
# Process donor info to segregate control group
control_group_df = donor_info[donor_info['act_demented'] == 'No Dementia']
dementia_group_df = donor_info[donor_info['act_demented'] != 'No Dementia']

# Get donor ids
control_ids = control_group_df['donor_id']
dementia_ids = dementia_group_df['donor_id']

# Assign condition to sample data
samples['Condition'] = samples['donor_id'].apply(lambda x: 'control' if x in control_ids.values else 'dementia')
samples

,rnaseq_profile_id,donor_id,donor_name,specimen_id,specimen_name,rna_well_id,polygon_id,structure_id,structure_acronym,structure_color,structure_name,hemisphere,Condition
0,488395315,309335467,H14.09.030,309357843,H14.09.030.TCx.01,395325172,320817998,10235,TCx,#ebbfd0,temporal neocortex,left,control
1,496100277,309335441,H14.09.004,309357624,H14.09.004.PCx.01,320630866,310967169,10557,FWM,#f2f1f0,white matter of forebrain,right,control
2,496100278,309335438,H14.09.001,309357596,H14.09.001.PCx.01,320630834,310790571,10557,FWM,#f2f1f0,white matter of forebrain,left,control
3,496100279,309335438,H14.09.001,309357599,H14.09.001.TCx.01,320630838,310790522,10235,TCx,#ebbfd0,temporal neocortex,left,control
4,496100281,309335439,H14.09.002,309357603,H14.09.002.HIP.01,320630842,310790372,10294,HIP,#bfb5d5,hippocampus (hippocampal formation),right,dementia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,496100667,467056391,H15.09.103,467179071,H15.09.103.TCx.01,482655826,480366830,10235,TCx,#ebbfd0,temporal neocortex,right,control
373,496100669,467056391,H15.09.103,467179068,H15.09.103.PCx.01,482655822,480363830,10557,FWM,#f2f1f0,white matter of forebrain,right,control
374,496100670,467056406,H15.09.107,467179104,H15.09.107.TCx.01,482655780,480363840,10235,TCx,#ebbfd0,temporal neocortex,right,dementia
375,496100671,467056391,H15.09.103,467179065,H15.09.103.HIP.01,482655820,480366825,10294,HIP,#bfb5d5,hippocampus (hippocampal formation),right,control


### Grab unique donor ids for proper data split

In [8]:
donor_ids = samples.donor_id.unique()

### 70-15-15 Train-Validate-Test donor_id split


In [9]:
# 70, 15, 15 Train, Validate, Test split
np.random.seed(42)
train_ids = np.random.choice(donor_ids, int(np.ceil(len(donor_ids) * 0.7)))
test_ids = np.setdiff1d(donor_ids, train_ids)
validate_ids = np.random.choice(test_ids, int(np.ceil(len(test_ids) * 0.5)))
test_ids = np.setdiff1d(test_ids, validate_ids)
len(train_ids), len(test_ids), len(validate_ids)

(75, 30, 27)

### Check condition distribution

In [10]:
samples[samples['donor_id'].isin(train_ids)]['Condition'].value_counts()

Condition
control     107
dementia     83
Name: count, dtype: int64

In [11]:
samples[samples['donor_id'].isin(validate_ids)]['Condition'].value_counts()

Condition
dementia    48
control     32
Name: count, dtype: int64

In [12]:
samples[samples['donor_id'].isin(test_ids)]['Condition'].value_counts()

Condition
control     58
dementia    49
Name: count, dtype: int64

### Create ML dataframe

In [13]:
counts = pd.read_pickle('../data/interim/PyDeseq2/ct_matrix.pkl')
top_genes = pd.read_pickle('../data/interim/PyDeseq2/top_bottom_ten_sigs.pkl')
lv_genes = pd.read_pickle('../data/interim/lv_genes')


In [31]:
print('Before dropping duplicates:',len(pd.concat((top_genes.gene_id, lv_genes.iloc[:,0]))))
print('After dropping duplicates:', len(pd.concat((top_genes.gene_id, lv_genes.iloc[:,0])).drop_duplicates()))

Before dropping duplicates: 40
After dropping duplicates: 32


In [34]:
gene_list = pd.concat((top_genes.gene_id, lv_genes.iloc[:,0])).drop_duplicates().values

In [35]:
# Filter by top genes (top and bottom 10)
ml_df = counts[counts.index.isin(gene_list)].copy()
ml_df

rnaseq_profile_id,488395315,496100277,496100278,496100279,496100281,496100283,496100284,496100285,496100287,496100288,...,496100661,496100663,496100664,496100665,496100666,496100667,496100669,496100670,496100671,496100672
gene_id_mapped,,,,,,,,,,,,,,,,,,,,,
499309627,21,85,83,44,40,34,39,32,26,27,...,43,45,24,20,17,53,52,26,56,40
499315162,51,143,275,70,112,80,57,45,28,100,...,76,95,151,28,105,87,78,66,117,39
499315537,13,22,14,49,28,27,48,44,14,48,...,70,43,51,9,24,103,103,11,124,41
499315843,18,13,6,12,5,25,18,15,20,19,...,26,12,11,39,20,23,29,35,11,10
499317231,31,15,30,42,55,38,35,62,37,39,...,64,89,47,23,22,73,42,48,46,41
499317260,4,3,6,10,4,4,10,10,6,4,...,14,13,5,5,6,8,4,0,11,7
499319747,7,5,2,13,3,9,11,16,5,6,...,39,24,8,5,3,39,26,6,21,10
499322427,638,216,184,806,435,222,287,713,491,513,...,702,1357,1418,308,976,1059,681,512,648,742
499323096,102,235,205,136,244,204,191,132,157,192,...,117,106,197,261,225,129,146,250,169,183


In [36]:
# Transform and apply conditions (y_label)
ml_df = ml_df.T.reset_index().rename_axis(None, axis = 1)
ml_df['Condition'] = samples['Condition']
ml_df


,rnaseq_profile_id,499309627,499315162,499315537,499315843,499317231,499317260,499319747,499322427,499323096,...,499336992,499342049,499343767,499343769,499347240,499348546,499348654,499350441,499352783,Condition
0,488395315,21,51,13,18,31,4,7,638,102,...,548,98,14,30,23,497,29,60,30,control
1,496100277,85,143,22,13,15,3,5,216,235,...,374,100,2,3,24,664,34,100,12,control
2,496100278,83,275,14,6,30,6,2,184,205,...,274,90,1,5,7,779,32,54,13,control
3,496100279,44,70,49,12,42,10,13,806,136,...,457,105,8,6,5,575,16,31,70,control
4,496100281,40,112,28,5,55,4,3,435,244,...,651,136,9,11,13,891,30,37,16,dementia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,496100667,53,87,103,23,73,8,39,1059,129,...,302,83,5,7,6,543,39,32,87,control
373,496100669,52,78,103,29,42,4,26,681,146,...,333,97,4,6,9,532,24,42,82,control
374,496100670,26,66,11,35,48,0,6,512,250,...,538,198,19,9,16,1101,55,54,28,dementia
375,496100671,56,117,124,11,46,11,21,648,169,...,308,128,2,6,10,575,26,53,23,control


In [37]:
#samples (rna_profile_ids) by donor data splt
train_samples = samples[samples['donor_id'].isin(train_ids)]['rnaseq_profile_id']
val_samples = samples[samples['donor_id'].isin(validate_ids)]['rnaseq_profile_id']
test_samples = samples[samples['donor_id'].isin(test_ids)]['rnaseq_profile_id']

len(train_samples), len(val_samples), len(test_samples)

(190, 80, 107)

In [38]:
# Now can filter by train, val, test, split
train_df = ml_df[ml_df['rnaseq_profile_id'].isin(train_samples)].drop(columns = 'rnaseq_profile_id')
val_df = ml_df[ml_df['rnaseq_profile_id'].isin(val_samples)].drop(columns = 'rnaseq_profile_id')
test_df = ml_df[ml_df['rnaseq_profile_id'].isin(test_samples)].drop(columns = 'rnaseq_profile_id')


In [39]:
# quick check
train_df

,499309627,499315162,499315537,499315843,499317231,499317260,499319747,499322427,499323096,499323166,...,499336992,499342049,499343767,499343769,499347240,499348546,499348654,499350441,499352783,Condition
1,85,143,22,13,15,3,5,216,235,362,...,374,100,2,3,24,664,34,100,12,control
2,83,275,14,6,30,6,2,184,205,297,...,274,90,1,5,7,779,32,54,13,control
3,44,70,49,12,42,10,13,806,136,466,...,457,105,8,6,5,575,16,31,70,control
4,40,112,28,5,55,4,3,435,244,545,...,651,136,9,11,13,891,30,37,16,dementia
5,34,80,27,25,38,4,9,222,204,725,...,541,130,10,21,17,873,44,49,55,dementia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,24,151,51,11,47,5,8,1418,197,519,...,508,90,4,10,12,681,62,72,41,dementia
372,53,87,103,23,73,8,39,1059,129,526,...,302,83,5,7,6,543,39,32,87,control
373,52,78,103,29,42,4,26,681,146,508,...,333,97,4,6,9,532,24,42,82,control
375,56,117,124,11,46,11,21,648,169,620,...,308,128,2,6,10,575,26,53,23,control


## Base Models

In [40]:
X_train = train_df.drop(columns='Condition')
y_train = train_df['Condition']

X_val = val_df.drop(columns='Condition')
y_val = val_df['Condition']

# Scale data and transform data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_= scaler.transform(X_train)
X_val_= scaler.transform(X_val)

y_train = y_train.apply(lambda x: 1 if x == 'dementia' else 0)
y_val = y_val.apply(lambda x: 1 if  x=='dementia' else 0)

In [41]:
dummy = DummyClassifier(strategy= 'most_frequent', random_state=42)
dummy.fit(X_train_, y_train)
preds = dummy.predict(X_val_)
score = classification_report(y_val, preds)
print(score)

              precision    recall  f1-score   support

           0       0.40      1.00      0.57        32
           1       0.00      0.00      0.00        48

    accuracy                           0.40        80
   macro avg       0.20      0.50      0.29        80
weighted avg       0.16      0.40      0.23        80



/opt/homebrew/Caskroom/miniforge/base/envs/mads/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/mads/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/mads/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

In [42]:
reg = LogisticRegression(random_state=42)
reg.fit(X_train_, y_train)
preds = reg.predict(X_val_)
score = classification_report(y_val, preds)
print(score)

              precision    recall  f1-score   support

           0       0.59      0.72      0.65        32
           1       0.78      0.67      0.72        48

    accuracy                           0.69        80
   macro avg       0.69      0.69      0.68        80
weighted avg       0.70      0.69      0.69        80



In [43]:
tree = DecisionTreeClassifier(random_state = 42)
tree.fit(X_train_, y_train)
preds = tree.predict(X_val_)
score = classification_report(y_val, preds)
print(score)

              precision    recall  f1-score   support

           0       0.53      0.81      0.64        32
           1       0.81      0.52      0.63        48

    accuracy                           0.64        80
   macro avg       0.67      0.67      0.64        80
weighted avg       0.70      0.64      0.64        80



In [44]:
forest = RandomForestClassifier(random_state= 42)
forest.fit(X_train_, y_train)
preds = forest.predict(X_val_)
score = classification_report(y_val, preds)
print(score)

              precision    recall  f1-score   support

           0       0.56      0.78      0.65        32
           1       0.80      0.58      0.67        48

    accuracy                           0.66        80
   macro avg       0.68      0.68      0.66        80
weighted avg       0.70      0.66      0.66        80



In [45]:
linear_svm = SVC(kernel='linear', random_state=42)
linear_svm.fit(X_train_, y_train)
preds = linear_svm.predict(X_val_)
score = classification_report(y_val, preds)
print(score)

              precision    recall  f1-score   support

           0       0.58      0.69      0.63        32
           1       0.76      0.67      0.71        48

    accuracy                           0.68        80
   macro avg       0.67      0.68      0.67        80
weighted avg       0.69      0.68      0.68        80



In [46]:
rbf_svm = SVC(kernel='rbf', random_state=42)
rbf_svm.fit(X_train_, y_train)
preds = rbf_svm.predict(X_val_)
score = classification_report(y_val, preds)
print(score)

              precision    recall  f1-score   support

           0       0.62      0.78      0.69        32
           1       0.82      0.69      0.75        48

    accuracy                           0.73        80
   macro avg       0.72      0.73      0.72        80
weighted avg       0.74      0.72      0.73        80



In [47]:
boost = GradientBoostingClassifier(random_state=42)
boost.fit(X_train_, y_train)
preds = boost.predict(X_val_)
score = classification_report(y_val, preds)
print(score)

              precision    recall  f1-score   support

           0       0.61      0.78      0.68        32
           1       0.82      0.67      0.74        48

    accuracy                           0.71        80
   macro avg       0.72      0.72      0.71        80
weighted avg       0.74      0.71      0.72        80



In [48]:

gauss = GaussianNB()
gauss.fit(X_train_, y_train)
preds = gauss.predict(X_val_)
score = classification_report(y_val, preds)
print(score)


              precision    recall  f1-score   support

           0       0.74      0.81      0.78        32
           1       0.87      0.81      0.84        48

    accuracy                           0.81        80
   macro avg       0.80      0.81      0.81        80
weighted avg       0.82      0.81      0.81        80



In [49]:
rbf_svm = SVC(kernel='rbf', random_state=42, probability= True)
rbf_svm.fit(X_train_, y_train)
preds = rbf_svm.predict_proba(X_val_)
score = average_precision_score(y_val, preds[:,1])
print(score)


0.8583377221947985


In [50]:
forest = RandomForestClassifier(random_state= 42)
forest.fit(X_train_, y_train)
preds = forest.predict_proba(X_val_)
score = average_precision_score(y_val, preds[:,1])
print(score)

0.769193004836744


### Use AVERAGE Precision score to evaluate several models


In [65]:
# Use AVERAGE Precision score
models = [
    LogisticRegression(random_state=42),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42),
    SVC(kernel="linear", random_state=42, probability=True),
    SVC(kernel="rbf", random_state=42, probability=True),
    GaussianNB(),
    GradientBoostingClassifier(random_state=42),
    DummyClassifier(strategy= 'most_frequent', random_state=42),
]
model_names = ["Log_Reg", "DT", "RF", 'SVM_linear', "SVM_radial", "GaussianNB", 'Gradient_boosted', 'Dummy_most_freq']

scores = {}
saved_models = {}
for name, model in zip(model_names, models):
    model.fit(X_train_, y_train)
    saved_models[name] = model
    preds = model.predict_proba(X_val_)
    score = average_precision_score(y_val, preds[:, 1])
    scores[name] = score

pd.Series(scores).sort_values(ascending=False).round(3)

GaussianNB          0.941
SVM_radial          0.858
Log_Reg             0.820
SVM_linear          0.815
Gradient_boosted    0.803
RF                  0.769
DT                  0.708
Dummy_most_freq     0.600
dtype: float64

In [ ]:
gene_data = pd.read_csv('../data/raw/gene_expression_matrix_2016-03-03/rows-genes.csv')

### Quick look at log reg coefficients


In [121]:
# Quick look at log reg coefficients
support_ids = X_train.columns.to_list()
log_reg_coefs = saved_models['Log_Reg'].coef_.tolist()[0]


In [138]:
gene_ids_ranking = (
pd.DataFrame.from_dict({k:v for k,v in zip(support_ids, log_reg_coefs)}, orient = 'index', columns= ['coef'])
.rename_axis('gene_id')
.reset_index()
.sort_values(by='coef', ascending= False)
)
gene_ids_ranking = gene_ids_ranking.merge(gene_data, on='gene_id')
gene_ids_ranking

,gene_id,coef,chromosome,gene_entrez_id,gene_symbol,gene_name
0,499348546,0.637636,19,23152,CIC,capicua transcriptional repressor
1,499343769,0.563935,16,105371409,LOC105371409,uncharacterized LOC105371409
2,499336992,0.547440,12,5426,POLE,"polymerase (DNA directed), epsilon, catalytic ..."
3,499350441,0.487532,20,140876,FAM65C,"family with sequence similarity 65, member C"
4,499315843,0.452432,4,285489,DOK7,docking protein 7
5,499317231,0.417990,4,4085,MAD2L1,MAD2 mitotic arrest deficient-like 1 (yeast)
6,499343767,0.341436,16,101927793,LOC101927793,uncharacterized LOC101927793
7,499323096,0.302684,7,79778,MICALL2,MICAL-like 2
8,499329195,0.240802,9,85301,COL27A1,"collagen, type XXVII, alpha 1"
9,499334626,0.218078,12,6539,SLC6A12,solute carrier family 6 (neurotransmitter tran...


### Quick look at DT feature performance


In [126]:
# Quick look at DT feature performance
saved_models['DT'].feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00679705, 0.        , 0.10974137, 0.01604549, 0.        ,
       0.        , 0.01944908, 0.01011498, 0.26058385, 0.        ,
       0.03554424, 0.02726113, 0.        , 0.04936825, 0.03697756,
       0.02106588, 0.        , 0.        , 0.03409347, 0.02852532,
       0.        , 0.07655832, 0.10158606, 0.        , 0.        ,
       0.16628796, 0.        ])

In [127]:
# RFE with decision tree
estimator = models[1]
selector = RFE(estimator)
selector = selector.fit(X_train_, y_train)
print(selector.support_)
print(selector.ranking_)

[False False False False False  True False  True False False False  True
  True  True  True  True False False  True  True  True  True False  True
 False False  True  True False  True  True False]
[17 14 12  9 13  1  6  1  8  5  3  1  1  1  1  1 16 11  1  1  1  1  2  1
 10  4  1  1  7  1  1 15]


In [134]:
#Top genes
# support_ids = X_train.columns[selector.support_].to_list()
support_ranking = saved_models['DT'].feature_importances_[selector.support_]

gene_ids_ranking = (
pd.DataFrame.from_dict({k:v for k,v in zip(support_ids, support_ranking)}, orient = 'index', columns= ['DT_ranking'])
.rename_axis('gene_id')
.reset_index()
.sort_values(by='DT_ranking', ascending= False)
)


In [137]:
gene_data_with_ranking = gene_ids_ranking.merge(gene_data, on='gene_id')
gene_data_with_ranking = gene_data_with_ranking[gene_data_with_ranking['DT_ranking'] > 0]
gene_data_with_ranking

,gene_id,DT_ranking,chromosome,gene_entrez_id,gene_symbol,gene_name
0,499317231,0.260584,4,4085,MAD2L1,MAD2 mitotic arrest deficient-like 1 (yeast)
1,499330118,0.166288,10,100130992,LOC100130992,uncharacterized LOC100130992
2,499315162,0.109741,3,100505385,IQCJ-SCHIP1,IQCJ-SCHIP1 readthrough
3,499328366,0.101586,9,100113421,LOC100113421,fibroblast growth factor 7 pseudogene
4,499328351,0.076558,9,728433,LOC728433,fibroblast growth factor 7 pseudogene
5,499322427,0.049368,6,9729,KIAA0408,KIAA0408
6,499323096,0.036978,7,79778,MICALL2,MICAL-like 2
7,499319747,0.035544,5,101926975,LOC101926975,uncharacterized LOC101926975
8,499326780,0.034093,8,51050,PI15,peptidase inhibitor 15
9,499323166,0.021066,7,84629,TNRC18,trinucleotide repeat containing 18
